# CHARTEVENTS
CHARTEVENTS.csv is a 35GB file after uncompress. It is split into 100,000 line chunks for easier uploading. The first chunk is loaded for analysis. 

In [ ]:
# Uncompress data file
#!gunzip MIMIC-data/CHARTEVENTS.csv.gz

In [ ]:
# Split file into 100,000 line chunks


In [1]:
import os
import numpy as np
import pandas as pd

In [7]:
# Load first 100,000 rows of CHARTEVENTS for analysis
df_charts = pd.read_csv('1/xaa', header=None, names=['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'ITEMID', 'CHARTTIME',
       'STORETIME', 'CGID', 'VALUE', 'VALUENUM', 'VALUEUOM', 'WARNING',
       'ERROR', 'RESULTSTATUS', 'STOPPED'])

In [9]:
df_charts.head(10)

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED
65536,61911,502,116367,209737.0,220052,2143-10-30 20:00:00,2143-10-30 21:44:00,17574,124.00,124.00,mmHg,0,0,NaN,NaN
65537,61912,502,116367,209737.0,220210,2143-10-30 20:00:00,2143-10-30 21:44:00,17574,24.00,24.00,insp/min,0,0,NaN,NaN
65538,61913,502,116367,209737.0,220277,2143-10-30 20:00:00,2143-10-30 21:44:00,17574,93.00,93.00,%,0,0,NaN,NaN
65539,61914,502,116367,209737.0,223762,2143-10-30 20:00:00,2143-10-30 21:43:00,17574,37.70,37.70,?C,0,0,NaN,NaN
65540,61915,502,116367,209737.0,220734,2143-10-30 20:40:00,2143-10-30 22:48:00,20889,6.00,6.00,units,0,0,NaN,NaN
65541,61916,502,116367,209737.0,227471,2143-10-30 20:40:00,2143-10-30 22:48:00,20889,1.03,1.03,NaN,0,0,NaN,NaN
65542,61917,502,116367,209737.0,220045,2143-10-30 21:00:00,2143-10-30 21:44:00,17574,110.00,110.00,bpm,0,0,NaN,NaN
65543,61918,502,116367,209737.0,220050,2143-10-30 21:00:00,2143-10-30 21:44:00,17574,146.00,146.00,mmHg,0,0,NaN,NaN
65544,61919,502,116367,209737.0,220051,2143-10-30 21:00:00,2143-10-30 21:44:00,17574,89.00,89.00,mmHg,0,0,NaN,NaN
65545,61920,502,116367,209737.0,220052,2143-10-30 21:00:00,2143-10-30 21:44:00,17574,107.00,107.00,mmHg,0,0,NaN,NaN


In [16]:
df_charts.groupby('ITEMID').count()

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED
ITEMID,,,,,,,,,,,,,,
220045,8089,8089,8089,8062,8089,8089,8089,8089,8089,8089,8089,8089,0,0
220046,962,962,962,960,962,962,962,962,962,962,962,962,0,0
220047,957,957,957,955,957,957,957,957,957,957,957,957,0,0
220050,2581,2581,2581,2554,2581,2581,2581,2581,2581,2581,2581,2581,0,0
220051,2581,2581,2581,2554,2581,2581,2581,2581,2581,2581,2581,2581,0,0
220052,2586,2586,2586,2559,2586,2586,2586,2586,2586,2586,2586,2586,0,0
220056,286,286,286,284,286,286,286,286,286,286,286,286,0,0
220058,287,287,287,285,287,287,287,287,287,287,287,287,0,0
220059,48,48,48,48,48,48,48,48,48,48,48,48,0,0


## Cross-referencing D_ITEMS.csv
To understand what each ITEMID represents, cross-reference the D_ITEMS.csv dictionary.


In [ ]:
df_Ditems = pd.read_csv()

In [17]:
# heart rate: ITEMID = 220045
df_charts[df_charts['ITEMID']==220045]

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED
65542,61917,502,116367,209737.0,220045,2143-10-30 21:00:00,2143-10-30 21:44:00,17574,110.0,110.0,bpm,0,0,NaN,NaN
65551,62908,502,116367,209737.0,220045,2143-11-02 06:01:00,2143-11-02 06:21:00,20855,104.0,104.0,bpm,0,0,NaN,NaN
65553,62910,502,116367,209737.0,220045,2143-11-02 07:00:00,2143-11-02 08:48:00,15922,94.0,94.0,bpm,0,0,NaN,NaN
65583,62940,502,116367,209737.0,220045,2143-11-02 08:00:00,2143-11-02 08:48:00,15922,90.0,90.0,bpm,0,0,NaN,NaN
65609,62966,502,116367,209737.0,220045,2143-11-02 09:00:00,2143-11-02 09:04:00,15922,92.0,92.0,bpm,0,0,NaN,NaN
65610,66517,605,115545,248569.0,220045,2197-11-10 01:46:00,2197-11-10 01:52:00,16436,71.0,71.0,bpm,0,0,NaN,NaN
65617,66524,605,115545,248569.0,220045,2197-11-10 01:47:00,2197-11-10 01:52:00,16436,71.0,71.0,bpm,0,0,NaN,NaN
65624,66531,605,115545,248569.0,220045,2197-11-10 01:48:00,2197-11-10 01:52:00,16436,71.0,71.0,bpm,0,0,NaN,NaN
65631,66538,605,115545,248569.0,220045,2197-11-10 01:49:00,2197-11-10 01:52:00,16436,72.0,72.0,bpm,0,0,NaN,NaN
65638,66545,605,115545,248569.0,220045,2197-11-10 01:50:00,2197-11-10 01:52:00,16436,71.0,71.0,bpm,0,0,NaN,NaN


In [73]:
# BP systolic: ITEMID = 220050
df_charts[df_charts['ITEMID']==220050].head()

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED
65543,61918,502,116367,209737.0,220050,2143-10-30 21:00:00,2143-10-30 21:44:00,17574,146.0,146.0,mmHg,0,0,NaN,NaN
65547,62904,502,116367,209737.0,220050,2143-11-02 06:00:00,2143-11-02 06:21:00,20855,160.0,160.0,mmHg,0,0,NaN,NaN
65554,62911,502,116367,209737.0,220050,2143-11-02 07:00:00,2143-11-02 08:48:00,15922,152.0,152.0,mmHg,0,0,NaN,NaN
65586,62943,502,116367,209737.0,220050,2143-11-02 08:00:00,2143-11-02 08:48:00,15922,151.0,151.0,mmHg,0,0,NaN,NaN
65611,66518,605,115545,248569.0,220050,2197-11-10 01:46:00,2197-11-10 01:52:00,16436,88.0,88.0,mmHg,0,0,NaN,NaN


In [79]:
# BP diastolic: ITEMID = 220051
df_charts[df_charts['ITEMID']==220051].count()

ROW_ID          2581
SUBJECT_ID      2581
HADM_ID         2581
ICUSTAY_ID      2554
ITEMID          2581
CHARTTIME       2581
STORETIME       2581
CGID            2581
VALUE           2581
VALUENUM        2581
VALUEUOM        2581
WARNING         2581
ERROR           2581
RESULTSTATUS       0
STOPPED            0
dtype: int64

In [77]:
# Temperature F: ITEMID = 223761
df_charts[df_charts['ITEMID']==223761].count()

ROW_ID          2060
SUBJECT_ID      2060
HADM_ID         2060
ICUSTAY_ID      2051
ITEMID          2060
CHARTTIME       2060
STORETIME       2060
CGID            2060
VALUE           2060
VALUENUM        2060
VALUEUOM        2060
WARNING         2060
ERROR           2060
RESULTSTATUS       0
STOPPED            0
dtype: int64

In [82]:
# White blood count (WBC): ITEMID = 220546
df_charts[df_charts['ITEMID']==220546].head()

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED
65766,65220,533,100009,253656.0,220546,2162-05-17 18:30:00,2162-05-17 18:52:00,20889,17.3,17.3,K/uL,1,0,NaN,NaN
65812,62065,502,116367,209737.0,220546,2143-10-31 04:43:00,2143-10-31 05:21:00,20889,36.1,36.1,K/uL,1,0,NaN,NaN
66769,64671,523,196271,227379.0,220546,2141-12-21 01:54:00,2141-12-21 02:17:00,20889,15.4,15.4,K/uL,1,0,NaN,NaN
67147,63254,502,116367,209737.0,220546,2143-11-03 02:23:00,2143-11-03 02:53:00,20889,25.5,25.5,K/uL,1,0,NaN,NaN
67447,66644,605,115545,248569.0,220546,2197-11-10 03:08:00,2197-11-10 03:59:00,20889,10.3,10.3,K/uL,0,0,NaN,NaN


### Input selection from Chart events
The following chart events will be used as input:
* Heart rate: ITEMID = 220045
* BP systolic: ITEMID = 220050
* BP diastolic: ITEMID = 220051
* Temperature F: ITEMID = 223761
* White blood count (WBC): ITEMID = 220546

The item codes are retrieved from D_ITEMS.csv

## Finding min, max of chart events
The following function summarizes the maximum and minimum values of a chart reading specified, and write it into a csv file.
The intent is to do so for all the split files of CHARTEVENTS_trim.csv.

In [62]:
def chart_min_max(df, itemIDname, itemIDvalue, groupbycol, outputfile):
    df_max = pd.DataFrame(df[df['ITEMID']==itemIDvalue].groupby(groupbycol).VALUENUM.max())
    df_min = pd.DataFrame(df[df['ITEMID']==itemIDvalue].groupby(groupbycol).VALUENUM.min())
    df_out=pd.merge(df_max, df_min, how='left', on=groupbycol)
    df_out.to_csv(outputfile, header=['MAX'+itemIDname,'MIN'+itemIDname])

In [64]:
chart_min_max(df_charts, 'HR',220045,'HADM_ID','HR.csv')

In [65]:
chart_min_max(df_charts, 'BPsys',220050,'HADM_ID','BPsys.csv')

In [66]:
chart_min_max(df_charts, 'BPdia',220051,'HADM_ID','BPdia.csv')

In [71]:
df_charts[df_charts['ITEMID']==223761]

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED
65594,62951,502,116367,209737.0,223761,2143-11-02 08:00:00,2143-11-02 08:48:00,15922,98.9,98.9,?F,0,0,NaN,NaN
65733,64204,518,185375,225925.0,223761,2111-07-08 08:00:00,2111-07-08 08:36:00,17250,97.9,97.9,?F,0,0,NaN,NaN
65851,64611,523,196271,227379.0,223761,2141-12-20 20:00:00,2141-12-20 20:15:00,14576,99.1,99.1,?F,0,0,NaN,NaN
65892,64652,523,196271,227379.0,223761,2141-12-21 00:00:00,2141-12-21 00:07:00,14576,98.1,98.1,?F,0,0,NaN,NaN
65904,61474,502,116367,209737.0,223761,2143-10-29 20:00:00,2143-10-29 21:46:00,17574,99.2,99.2,?F,0,0,NaN,NaN
65945,69069,605,115545,248569.0,223761,2197-11-14 17:00:00,2197-11-14 17:29:00,16326,98.0,98.0,?F,0,0,NaN,NaN
65987,69111,605,115545,248569.0,223761,2197-11-14 20:00:00,2197-11-14 20:23:00,16273,96.7,96.7,?F,0,0,NaN,NaN
66253,63218,502,116367,209737.0,223761,2143-11-03 00:00:00,2143-11-03 01:13:00,14605,98.2,98.2,?F,0,0,NaN,NaN
66363,63621,502,116367,209737.0,223761,2143-11-03 20:00:00,2143-11-03 20:26:00,14605,97.7,97.7,?F,0,0,NaN,NaN
66390,66145,605,115545,248569.0,223761,2197-11-09 20:00:00,2197-11-09 22:54:00,16436,96.1,96.1,?F,0,0,NaN,NaN


In [40]:
df_HR = pd.DataFrame(df_charts[df_charts['ITEMID']==220045].groupby('HADM_ID').VALUENUM.max())

In [41]:
df_HR.describe()

,VALUENUM
count,67.000000
mean,106.314925
std,24.591993
min,63.000000
25%,87.000000
50%,104.000000
75%,121.000000
max,161.000000


In [44]:
df_HR2 = pd.DataFrame(df_charts[df_charts['ITEMID']==220045].groupby('HADM_ID').VALUENUM.min())

df_HR2.head()

In [48]:
df=pd.merge(df_HR, df_HR2, how='left', on='HADM_ID')

In [49]:
df.head()

,VALUENUM_x,VALUENUM_y
HADM_ID,,
100009,92.0,49.0
101322,92.0,31.0
102365,112.0,78.0
102607,125.0,69.0
104140,139.0,54.0


In [52]:
import matplotlib.pyplot as plt

In [57]:
df.to_csv('HR_maxmin.csv', header=['MAXHR','MINHR'])

## Alternative way to extract chart features
Using awk command to filter for specific itemID through all the chunks



In [ ]:
#!awk -F, '$6 == "220045" {print $4, $11 } ' x* > ../Heartrate_220045.csv
#!awk -F, '$6 == "220050" {print $4, $11 } ' x* > ../BPsys_220050.csv
#!awk -F, '$6 == "223761" {print $4, $11 } ' x* > ../BPdia_220051.csv
#!awk -F, '$6 == "223761" {print $4, $11 } ' x* > ../Temp_223761.csv
#!awk -F, '$6 == "220546" {print $4, $11 } ' x* > ../WBC_220546.csv

## Read in each file to summarize

In [2]:
import os
import numpy as np
import pandas as pd

In [41]:
data_dir = 'MIMIC-data'
df_HR = pd.read_csv(os.path.join(data_dir,'Heartrate_220045.csv'), sep=' ', names=['HADM_ID','ITEMID','HR'])
df_BPsys = pd.read_csv(os.path.join(data_dir,'BPsys_220050.csv'), sep=' ', names=['HADM_ID','ITEMID','BPsys'])
df_BPdia = pd.read_csv(os.path.join(data_dir,'BPdia_220051.csv'), sep=' ', names=['HADM_ID','ITEMID','BPdia'])
df_Temp = pd.read_csv(os.path.join(data_dir,'Temp_223761.csv'), sep=' ', names=['HADM_ID','Temp'])
df_WBC = pd.read_csv(os.path.join(data_dir,'WBC_220546.csv'), sep=' ', names=['HADM_ID','WBC'])

In [42]:
df_Temp.head()

,HADM_ID,Temp
0,116367,98.9
1,185375,97.9
2,196271,99.1
3,196271,98.1
4,116367,99.2


In [43]:
#drop extra columns
df_HR = df_HR.drop(columns={'ITEMID'})
df_BPsys = df_BPsys.drop(columns={'ITEMID'})
df_BPdia = df_BPdia.drop(columns={'ITEMID'})

In [44]:
def min_max(df, itemIDname, groupbycol):
    df_min = pd.DataFrame(df.groupby(groupbycol)[itemIDname].min())
    df_max = pd.DataFrame(df.groupby(groupbycol)[itemIDname].max())
    df_out=pd.merge(df_min, df_max, how='left', on=groupbycol)
    return df_out

In [45]:
df_HR_minmax = min_max(df_HR, 'HR', 'HADM_ID')

In [46]:
df_HR_minmax.head()

,HR_x,HR_y
HADM_ID,,
100001,75.0,131.0
100003,69.0,104.0
100009,49.0,92.0
100011,68.0,142.0
100012,67.0,109.0


In [47]:
df_BPsys_minmax = min_max(df_BPsys, 'BPsys', 'HADM_ID')
df_BPdia_minmax = min_max(df_BPdia, 'BPdia', 'HADM_ID')
df_Temp_minmax = min_max(df_Temp, 'Temp', 'HADM_ID')
df_WBC_minmax = min_max(df_WBC, 'WBC', 'HADM_ID')

In [49]:
df_datamerge = pd.merge(df_HR_minmax, df_BPsys_minmax, how='outer', on='HADM_ID')
df_datamerge = pd.merge(df_datamerge, df_BPdia_minmax, how='outer', on='HADM_ID')
df_datamerge = pd.merge(df_datamerge, df_Temp_minmax, how='outer', on='HADM_ID')
df_datamerge = pd.merge(df_datamerge, df_WBC_minmax, how='outer', on='HADM_ID')
df_datamerge.head()

,HR_x,HR_y,BPsys_x,BPsys_y,BPdia_x,BPdia_y,Temp_x,Temp_y,WBC_x,WBC_y
HADM_ID,,,,,,,,,,
100001,75.0,131.0,NaN,NaN,NaN,NaN,97.5,100.0,6.1,15.9
100003,69.0,104.0,NaN,NaN,NaN,NaN,95.4,98.2,12.7,14.2
100009,49.0,92.0,98.0,132.0,48.0,60.0,NaN,NaN,17.3,17.3
100011,68.0,142.0,95.0,186.0,43.0,127.0,97.5,102.7,7.6,17.3
100012,67.0,109.0,88.0,151.0,45.0,75.0,97.0,99.3,6.9,14.1


In [50]:
df_datamerge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20981 entries, 100001 to 199998
Data columns (total 10 columns):
HR_x       18868 non-null float64
HR_y       18868 non-null float64
BPsys_x    7028 non-null float64
BPsys_y    7028 non-null float64
BPdia_x    7026 non-null float64
BPdia_y    7026 non-null float64
Temp_x     18712 non-null float64
Temp_y     18712 non-null float64
WBC_x      20545 non-null float64
WBC_y      20545 non-null float64
dtypes: float64(10)
memory usage: 1.8 MB


## Extract patient info

In [51]:
df_patient = pd.read_csv(os.path.join(data_dir,'PATIENTS.csv'))

In [52]:
df_patient.head()

,ROW_ID,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG
0,234,249,F,2075-03-13 00:00:00,NaN,NaN,NaN,0
1,235,250,F,2164-12-27 00:00:00,2188-11-22 00:00:00,2188-11-22 00:00:00,NaN,1
2,236,251,M,2090-03-15 00:00:00,NaN,NaN,NaN,0
3,237,252,M,2078-03-06 00:00:00,NaN,NaN,NaN,0
4,238,253,F,2089-11-26 00:00:00,NaN,NaN,NaN,0


In [53]:
# extract only needed columns
df_patient = pd.read_csv(os.path.join(data_dir,'PATIENTS.csv'), 
                         usecols=['SUBJECT_ID','GENDER','DOB'])

In [60]:
df_patient.head()

,SUBJECT_ID,GENDER,DOB
0,249,F,2075-03-13 00:00:00
1,250,F,2164-12-27 00:00:00
2,251,M,2090-03-15 00:00:00
3,252,M,2078-03-06 00:00:00
4,253,F,2089-11-26 00:00:00


## Extract admission info

In [1]:
df_adm = pd.read_csv(os.path.join(data_dir,'ADMISSIONS_TRIM.csv'))

NameError: name 'pd' is not defined

In [68]:
df_adm.head()

,Unnamed: 0,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,ADMISSION_TYPE,SEPSIS_FLAG,LENGTH_OF_STAY
0,0,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,EMERGENCY,False,1 days 03:28:00.000000000
1,2,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,EMERGENCY,False,6 days 18:26:00.000000000
2,3,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,EMERGENCY,False,2 days 20:34:00.000000000
3,4,25,129635,2160-11-02 02:06:00,2160-11-05 14:55:00,EMERGENCY,False,3 days 12:49:00.000000000
4,5,26,197661,2126-05-06 15:16:00,2126-05-13 15:00:00,EMERGENCY,False,6 days 23:44:00.000000000


## Merge data together

In [69]:
df_adm = pd.merge(df_adm,df_patient, how='left', on='SUBJECT_ID')

In [70]:
df_adm.head()

,Unnamed: 0,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,ADMISSION_TYPE,SEPSIS_FLAG,LENGTH_OF_STAY,GENDER,DOB
0,0,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,EMERGENCY,False,1 days 03:28:00.000000000,F,2131-05-07 00:00:00
1,2,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,EMERGENCY,False,6 days 18:26:00.000000000,M,2082-07-17 00:00:00
2,3,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,EMERGENCY,False,2 days 20:34:00.000000000,M,2100-05-31 00:00:00
3,4,25,129635,2160-11-02 02:06:00,2160-11-05 14:55:00,EMERGENCY,False,3 days 12:49:00.000000000,M,2101-11-21 00:00:00
4,5,26,197661,2126-05-06 15:16:00,2126-05-13 15:00:00,EMERGENCY,False,6 days 23:44:00.000000000,M,2054-05-04 00:00:00


In [71]:
df_adm = pd.merge(df_adm,df_datamerge, how='left', on='HADM_ID')

In [72]:
df_adm.head()

,Unnamed: 0,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,ADMISSION_TYPE,SEPSIS_FLAG,LENGTH_OF_STAY,GENDER,DOB,HR_x,HR_y,BPsys_x,BPsys_y,BPdia_x,BPdia_y,Temp_x,Temp_y,WBC_x,WBC_y
0,0,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,EMERGENCY,False,1 days 03:28:00.000000000,F,2131-05-07 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,EMERGENCY,False,6 days 18:26:00.000000000,M,2082-07-17 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,EMERGENCY,False,2 days 20:34:00.000000000,M,2100-05-31 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,25,129635,2160-11-02 02:06:00,2160-11-05 14:55:00,EMERGENCY,False,3 days 12:49:00.000000000,M,2101-11-21 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,26,197661,2126-05-06 15:16:00,2126-05-13 15:00:00,EMERGENCY,False,6 days 23:44:00.000000000,M,2054-05-04 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [75]:
#df_adm.drop(columns={'Unnamed:0'})

In [76]:
df_adm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43407 entries, 0 to 43406
Data columns (total 20 columns):
Unnamed: 0        43407 non-null int64
SUBJECT_ID        43407 non-null int64
HADM_ID           43407 non-null int64
ADMITTIME         43407 non-null object
DISCHTIME         43407 non-null object
ADMISSION_TYPE    43407 non-null object
SEPSIS_FLAG       43395 non-null object
LENGTH_OF_STAY    43407 non-null object
GENDER            43407 non-null object
DOB               43407 non-null object
HR_x              18868 non-null float64
HR_y              18868 non-null float64
BPsys_x           7028 non-null float64
BPsys_y           7028 non-null float64
BPdia_x           7026 non-null float64
BPdia_y           7026 non-null float64
Temp_x            18712 non-null float64
Temp_y            18712 non-null float64
WBC_x             20545 non-null float64
WBC_y             20545 non-null float64
dtypes: float64(10), int64(3), object(7)
memory usage: 7.0+ MB
